# Text Utilities

In [ ]:
#| default_exp text.utils

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import re
from typing import Dict
import inflect
from anyascii import anyascii

## TTS Cleaning & Normalization

In [ ]:
#| export
class TTSTextNormalizer:
    def __init__(self):
        # ABBREVIATIONS
        self.abbreviations_en = [
            (re.compile("\\b%s\\." % x[0], re.IGNORECASE), x[1])
            for x in [
                ("mrs", "misess"),
                ("mr", "mister"),
                ("dr", "doctor"),
                ("st", "saint"),
                ("co", "company"),
                ("jr", "junior"),
                ("maj", "major"),
                ("gen", "general"),
                ("drs", "doctors"),
                ("rev", "reverend"),
                ("lt", "lieutenant"),
                ("hon", "honorable"),
                ("sgt", "sergeant"),
                ("capt", "captain"),
                ("esq", "esquire"),
                ("ltd", "limited"),
                ("col", "colonel"),
                ("ft", "fort"),
            ]
        ]
        # NUMBERS
        self._inflect = inflect.engine()
        self._comma_number_re = re.compile(r"([0-9][0-9\,]+[0-9])")
        self._decimal_number_re = re.compile(r"([0-9]+\.[0-9]+)")
        self._currency_re = re.compile(r"(£|\$|¥)([0-9\,\.]*[0-9]+)")
        self._ordinal_re = re.compile(r"[0-9]+(st|nd|rd|th)")
        self._number_re = re.compile(r"-?[0-9]+")
        # TIME
        self._time_re = re.compile(
            r"""\b
            ((0?[0-9])|(1[0-1])|(1[2-9])|(2[0-3]))  # hours
            :
            ([0-5][0-9])                            # minutes
            \s*(a\\.m\\.|am|pm|p\\.m\\.|a\\.m|p\\.m)? # am/pm
            \b""",
            re.IGNORECASE | re.X,
        )
        # LANG CLEAN
        self._whitespace_re = re.compile(r"\s+")

    # NUMBERS
    def _remove_commas(self, m):
        return m.group(1).replace(",", "")

    def _expand_decimal_point(self, m):
        return m.group(1).replace(".", " point ")

    def __expand_currency(self, value: str, inflection: Dict[float, str]) -> str:
        parts = value.replace(",", "").split(".")
        if len(parts) > 2:
            return f"{value} {inflection[2]}"  # Unexpected format
        text = []
        integer = int(parts[0]) if parts[0] else 0
        if integer > 0:
            integer_unit = inflection.get(integer, inflection[2])
            text.append(f"{integer} {integer_unit}")
        fraction = int(parts[1]) if len(parts) > 1 and parts[1] else 0
        if fraction > 0:
            fraction_unit = inflection.get(fraction / 100, inflection[0.02])
            text.append(f"{fraction} {fraction_unit}")
        if len(text) == 0:
            return f"zero {inflection[2]}"
        return " ".join(text)

    def _expand_currency(self, m: "re.Match") -> str:
        currencies = {
            "$": {
                0.01: "cent",
                0.02: "cents",
                1: "dollar",
                2: "dollars",
            },
            "€": {
                0.01: "cent",
                0.02: "cents",
                1: "euro",
                2: "euros",
            },
            "£": {
                0.01: "penny",
                0.02: "pence",
                1: "pound sterling",
                2: "pounds sterling",
            },
            "¥": {
                # TODO rin
                0.02: "sen",
                2: "yen",
            },
        }
        unit = m.group(1)
        currency = currencies[unit]
        value = m.group(2)
        return self.__expand_currency(value, currency)

    def _expand_ordinal(self, m):
        return self._inflect.number_to_words(m.group(0))

    def _expand_number(self, m):
        num = int(m.group(0))
        if 1000 < num < 3000:
            if num == 2000:
                return "two thousand"
            if 2000 < num < 2010:
                return "two thousand " + self._inflect.number_to_words(num % 100)
            if num % 100 == 0:
                return self._inflect.number_to_words(num // 100) + " hundred"
            return self._inflect.number_to_words(num, andword="", zero="oh", group=2).replace(", ", " ")
        return self._inflect.number_to_words(num, andword="")

    def en_normalize_numbers(self, text):
        text = re.sub(self._comma_number_re, self._remove_commas, text)
        text = re.sub(self._currency_re, self._expand_currency, text)
        text = re.sub(self._decimal_number_re, self._expand_decimal_point, text)
        text = re.sub(self._ordinal_re, self._expand_ordinal, text)
        text = re.sub(self._number_re, self._expand_number, text)
        return text

    # TIME
    def _expand_num(self, n: int) -> str:
        return self._inflect.number_to_words(n)

    def _expand_time_english(self, match: "re.Match") -> str:
        hour = int(match.group(1))
        past_noon = hour >= 12
        time = []
        if hour > 12:
            hour -= 12
        elif hour == 0:
            hour = 12
            past_noon = True
        time.append(self._expand_num(hour))

        minute = int(match.group(6))
        if minute > 0:
            if minute < 10:
                time.append("oh")
            time.append(self._expand_num(minute))
        am_pm = match.group(7)
        if am_pm is None:
            time.append("p m" if past_noon else "a m")
        else:
            time.extend(list(am_pm.replace(".", "")))
        return " ".join(time)

    def expand_time_english(self, text: str) -> str:
        return re.sub(self._time_re, self._expand_time_english, text)

    # BASIC CLEANING
    def expand_abbreviations(self, text, lang="en"):
        if lang == "en":
            _abbreviations = self.abbreviations_en
        elif lang == "fr":
            pass
        for regex, replacement in _abbreviations:
            text = re.sub(regex, replacement, text)
        return text

    def lowercase(self, text):
        return text.lower()

    def collapse_whitespace(self, text):
        return re.sub(self._whitespace_re, " ", text).strip()

    def convert_to_ascii(self, text):
        return anyascii(text)

    def remove_aux_symbols(self, text):
        text = re.sub(r"[\<\>\(\)\[\]\"]+", "", text)
        return text

    def replace_symbols(self, text, lang="en"):
        text = text.replace(";", ",")
        text = text.replace("-", " ")
        text = text.replace(":", ",")
        if lang == "en":
            text = text.replace("&", " and ")
        elif lang == "fr":
            text = text.replace("&", " et ")
        elif lang == "pt":
            text = text.replace("&", " e ")
        return text
    
    # LANG CLEANERS

    def basic_cleaners(self, text):
        """Basic pipeline that lowercases and collapses whitespace without transliteration."""
        text = self.lowercase(text)
        text = self.collapse_whitespace(text)
        return text

    def transliteration_cleaners(self, text):
        """Pipeline for non-English text that transliterates to ASCII."""
        # text = convert_to_ascii(text)
        text = self.lowercase(text)
        text = self.collapse_whitespace(text)
        return text

    def basic_german_cleaners(self, text):
        """Pipeline for German text"""
        text = self.lowercase(text)
        text = self.collapse_whitespace(text)
        return text

    def basic_turkish_cleaners(self, text):
        """Pipeline for Turkish text"""
        text = text.replace("I", "ı")
        text = self.lowercase(text)
        text = self.collapse_whitespace(text)
        return text

    def english_cleaners(self, text):
        """Pipeline for English text, including number and abbreviation expansion."""
        # text = convert_to_ascii(text)
        text = self.lowercase(text)
        text = self.expand_time_english(text)
        text = self.en_normalize_numbers(text)
        text = self.expand_abbreviations(text)
        text = self.replace_symbols(text)
        text = self.remove_aux_symbols(text)
        text = self.collapse_whitespace(text)
        return text

    def phoneme_cleaners(self, text):
        """Pipeline for phonemes mode, including number and abbreviation expansion."""
        text = self.en_normalize_numbers(text)
        text = self.expand_abbreviations(text)
        text = self.replace_symbols(text)
        text = self.remove_aux_symbols(text)
        text = self.collapse_whitespace(text)
        return text

    def french_cleaners(self, text):
        """Pipeline for French text. There is no need to expand numbers, phonemizer already does that"""
        text = self.expand_abbreviations(text, lang="fr")
        text = self.lowercase(text)
        text = self.replace_symbols(text, lang="fr")
        text = self.remove_aux_symbols(text)
        text = self.collapse_whitespace(text)
        return text

    def portuguese_cleaners(self, text):
        """Basic pipeline for Portuguese text. There is no need to expand abbreviation and
        numbers, phonemizer already does that"""
        text = self.lowercase(text)
        text = self.replace_symbols(text, lang="pt")
        text = self.remove_aux_symbols(text)
        text = self.collapse_whitespace(text)
        return text

    def multilingual_cleaners(self, text):
        """Pipeline for multilingual text"""
        text = self.lowercase(text)
        text = self.replace_symbols(text, lang=None)
        text = self.remove_aux_symbols(text)
        text = self.collapse_whitespace(text)
        return text


In [ ]:
cleaner = TTSTextNormalizer()
print(cleaner.en_normalize_numbers("$350"))
print(cleaner.expand_time_english("12:05pm"))
print(cleaner.english_cleaners("Oh my dear! this is $5 too soon... It's 1:04 am!"))

three hundred fifty dollars
twelve oh five p m
oh my dear! this is five dollars too soon... it's one oh four a m!


In [ ]:
#| export
import collections
from enum import Enum
import six

In [ ]:
#| export

_DEF_PUNCS = ';:,.!?¡¿—…"«»“”'
_PUNC_IDX = collections.namedtuple("_punc_index", ["punc", "position"])

class PuncPosition(Enum):
    """Enum for the punctuations positions"""

    BEGIN = 0
    END = 1
    MIDDLE = 2
    ALONE = 3

class Punctuation:
    """Handle punctuations in text.

    Just strip punctuations from text or strip and restore them later.

    Args:
        puncs (str): The punctuations to be processed. Defaults to `_DEF_PUNCS`.

    Example:
        >>> punc = Punctuation()
        >>> punc.strip("This is. example !")
        'This is example'

        >>> text_striped, punc_map = punc.strip_to_restore("This is. example !")
        >>> ' '.join(text_striped)
        'This is example'

        >>> text_restored = punc.restore(text_striped, punc_map)
        >>> text_restored[0]
        'This is. example !'
    """

    def __init__(self, puncs: str = _DEF_PUNCS):
        self.puncs = puncs

    @staticmethod
    def default_puncs():
        """Return default set of punctuations."""
        return _DEF_PUNCS

    @property
    def puncs(self):
        return self._puncs

    @puncs.setter
    def puncs(self, value):
        if not isinstance(value, six.string_types):
            raise ValueError("[!] Punctuations must be of type str.")
        self._puncs = "".join(list(dict.fromkeys(list(value))))  # remove duplicates without changing the oreder
        self.puncs_regular_exp = re.compile(rf"(\s*[{re.escape(self._puncs)}]+\s*)+")

    def strip(self, text):
        """Remove all the punctuations by replacing with `space`.

        Args:
            text (str): The text to be processed.

        Example::

            "This is. example !" -> "This is example "
        """
        return re.sub(self.puncs_regular_exp, " ", text).rstrip().lstrip()

    def strip_to_restore(self, text):
        """Remove punctuations from text to restore them later.

        Args:
            text (str): The text to be processed.

        Examples ::

            "This is. example !" -> [["This is", "example"], [".", "!"]]

        """
        text, puncs = self._strip_to_restore(text)
        return text, puncs

    def _strip_to_restore(self, text):
        """Auxiliary method for Punctuation.preserve()"""
        matches = list(re.finditer(self.puncs_regular_exp, text))
        if not matches:
            return [text], []
        # the text is only punctuations
        if len(matches) == 1 and matches[0].group() == text:
            return [], [_PUNC_IDX(text, PuncPosition.ALONE)]
        # build a punctuation map to be used later to restore punctuations
        puncs = []
        for match in matches:
            position = PuncPosition.MIDDLE
            if match == matches[0] and text.startswith(match.group()):
                position = PuncPosition.BEGIN
            elif match == matches[-1] and text.endswith(match.group()):
                position = PuncPosition.END
            puncs.append(_PUNC_IDX(match.group(), position))
        # convert str text to a List[str], each item is separated by a punctuation
        splitted_text = []
        for idx, punc in enumerate(puncs):
            split = text.split(punc.punc)
            prefix, suffix = split[0], punc.punc.join(split[1:])
            splitted_text.append(prefix)
            # if the text does not end with a punctuation, add it to the last item
            if idx == len(puncs) - 1 and len(suffix) > 0:
                splitted_text.append(suffix)
            text = suffix
        return splitted_text, puncs

    @classmethod
    def restore(cls, text, puncs):
        """Restore punctuation in a text.

        Args:
            text (str): The text to be processed.
            puncs (List[str]): The list of punctuations map to be used for restoring.

        Examples ::

            ['This is', 'example'], ['.', '!'] -> "This is. example!"

        """
        return cls._restore(text, puncs, 0)

    @classmethod
    def _restore(cls, text, puncs, num):  # pylint: disable=too-many-return-statements
        """Auxiliary method for Punctuation.restore()"""
        if not puncs:
            return text

        # nothing have been phonemized, returns the puncs alone
        if not text:
            return ["".join(m.mark for m in puncs)]

        current = puncs[0]

        if current.position == PuncPosition.BEGIN:
            return cls._restore([current.punc + text[0]] + text[1:], puncs[1:], num)

        if current.position == PuncPosition.END:
            return [text[0] + current.punc] + cls._restore(text[1:], puncs[1:], num + 1)

        if current.position == PuncPosition.ALONE:
            return [current.mark] + cls._restore(text, puncs[1:], num + 1)

        # POSITION == MIDDLE
        if len(text) == 1:  # pragma: nocover
            # a corner case where the final part of an intermediate
            # mark (I) has not been phonemized
            return cls._restore([text[0] + current.punc], puncs[1:], num)

        return cls._restore([text[0] + current.punc + text[1]] + text[2:], puncs[1:], num)


In [ ]:
punc = Punctuation()
text = "This is. This is, example!"
print(punc.strip(text))
split_text, puncs = punc.strip_to_restore(text)
print(split_text, " ---- ", puncs)
restored_text = punc.restore(split_text, puncs)
print(restored_text)

This is This is example
['This is', 'This is', 'example']  ----  [_punc_index(punc='. ', position=<PuncPosition.MIDDLE: 2>), _punc_index(punc=', ', position=<PuncPosition.MIDDLE: 2>), _punc_index(punc='!', position=<PuncPosition.END: 1>)]
['This is. This is, example!']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()